<a href="https://colab.research.google.com/github/Faizullah565/Cloudanary-file/blob/master/url_link_img_Scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import requests
from bs4 import BeautifulSoup
from google.colab import files

# URL to scrape
url = "https://gcuf.edu.pk/"  # Replace with your target website

# File name for saving
file_name = "links.txt"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the webpage content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all links (anchor tags)
    links = [a['href'] for a in soup.find_all('a', href=True)]

    # Extract the first 50 unique links
    unique_links = list(set(links))[:50]

    # Save the links to a text file
    with open(file_name, "w") as file:
        for link in unique_links:
            file.write(link + "\n")

    print(f"File saved as {file_name}. Preparing for download...")

    # Use Colab's file download functionality
    files.download(file_name)
else:
    print(f"Failed to fetch the webpage. Status code: {response.status_code}")


File saved as links.txt. Preparing for download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from zipfile import ZipFile
from google.colab import files

def extract_images_from_links(output_folder, zip_file_name):
    # Allow user to upload a file containing links
    uploaded = files.upload()
    if len(uploaded) == 0:
        print("No file uploaded.")
        return

    links_file = list(uploaded.keys())[0]

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Read links from the uploaded file
    try:
        with open(links_file, 'r') as file:
            links = [line.strip() for line in file.readlines()]
    except Exception as e:
        print(f"Error reading file {links_file}: {e}")
        return

    # Iterate through each link and extract images
    for i, link in enumerate(links):
        if link.startswith("http"):
            try:
                response = requests.get(link)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    images = soup.find_all('img')

                    # Folder for each link's images
                    link_folder = os.path.join(output_folder, f'link_{i + 1}')
                    if not os.path.exists(link_folder):
                        os.makedirs(link_folder)

                    for j, img in enumerate(images):
                        img_url = img.get('src')
                        if img_url:
                            # Handle relative URLs
                            img_url = urljoin(link, img_url)

                            # Download the image
                            try:
                                img_response = requests.get(img_url, stream=True)
                                if img_response.status_code == 200:
                                    img_path = os.path.join(link_folder, f'image_{j + 1}.jpg')
                                    with open(img_path, 'wb') as img_file:
                                        for chunk in img_response.iter_content(1024):
                                            img_file.write(chunk)
                            except Exception as e:
                                print(f"Failed to download image {img_url}: {e}")
            except Exception as e:
                print(f"Failed to fetch images from {link}: {e}")

    # Create a ZIP file for download
    with ZipFile(zip_file_name, 'w') as zipf:
        for folder_name, subfolders, filenames in os.walk(output_folder):
            for filename in filenames:
                file_path = os.path.join(folder_name, filename)
                zipf.write(file_path, os.path.relpath(file_path, output_folder))

    print(f"Images have been saved and zipped as '{zip_file_name}'.")

    # Download the ZIP file
    files.download(zip_file_name)

# Configuration
output_folder = "images_from_links"  # Folder to save the images
zip_file_name = "images_from_links.zip"  # Name of the ZIP file
extract_images_from_links(output_folder, zip_file_name)


Saving links (6).txt to links (6).txt
Failed to download image http://ocw.vu.edu.pk/App_Themes/Default/images/banner/1.jpg: HTTPSConnectionPool(host='ocw.vu.edu.pk', port=443): Max retries exceeded with url: /App_Themes/Default/images/banner/1.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7d5d68ee00d0>, 'Connection to ocw.vu.edu.pk timed out. (connect timeout=None)'))
Failed to download image http://ocw.vu.edu.pk/App_Themes/Default/images/banner/2.jpg: ('Connection broken: IncompleteRead(5255 bytes read, 24699 more expected)', IncompleteRead(5255 bytes read, 24699 more expected))
Failed to download image http://ocw.vu.edu.pk/App_Themes/Default/Images/twitter_icon.gif: HTTPSConnectionPool(host='ocw.vu.edu.pk', port=443): Max retries exceeded with url: /App_Themes/Default/Images/twitter_icon.gif (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7d5d37181120>, 'Connection to ocw.vu.edu.pk timed out. (connect timeout=None)')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
# Install Selenium and other necessary libraries
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Configure Selenium WebDriver for Colab
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run browser in headless mode
chrome_options.add_argument('--no-sandbox')  # Required for Colab
chrome_options.add_argument('--disable-dev-shm-usage')  # Avoid memory issues

driver_path = "/usr/bin/chromedriver"
service = Service(driver_path)

def scrape_complete_website(base_url, output_folder="scraped_data"):
    # Initialize WebDriver
    driver = webdriver.Chrome(service=service, options=chrome_options)
    visited_urls = set()  # Track visited links
    urls_to_visit = [base_url]  # Start with the base URL

    # Create output folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    try:
        while urls_to_visit:
            current_url = urls_to_visit.pop(0)
            if current_url in visited_urls:
                continue

            print(f"Scraping: {current_url}")
            driver.get(current_url)
            visited_urls.add(current_url)

            # Save page content
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
            page_title = soup.title.string if soup.title else "untitled_page"
            safe_title = page_title.replace(" ", "_").replace("/", "_")
            file_path = os.path.join(output_folder, f"{safe_title}.html")
            with open(file_path, "w", encoding="utf-8") as file:
                file.write(page_source)

            # Extract all links from the current page
            anchors = soup.find_all("a", href=True)
            for anchor in anchors:
                href = anchor["href"]
                full_url = urljoin(current_url, href)
                if base_url in full_url and full_url not in visited_urls:
                    urls_to_visit.append(full_url)

            time.sleep(1)  # Be polite to the server

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()
        print(f"Scraping complete. Data saved in '{output_folder}'.")

# Define your target website URL
base_url = "https://gcuf.edu.pk/"  # Replace with the target website URL

# Scrape the website
scrape_complete_website(base_url)


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

WebDriverException: Message: Service /usr/bin/chromedriver unexpectedly exited. Status code was: 1


In [26]:
import requests
from bs4 import BeautifulSoup

# Function to get the HTML content of a webpage
def get_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve page: {url}")
        return None

# Function to scrape a single page
def scrape_page(url):
    html_content = get_page(url)
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')

        # Example: scraping all links from the page
        links = soup.find_all('a', href=True)
        for link in links:
            print(link['href'])

# Function to scrape the complete website, handling pagination
def scrape_complete_website(base_url):
    page_number = 1  # Start from page 1
    while True:
        page_url = f"{base_url}/page/{page_number}"  # Adjust URL structure to match pagination
        print(f"Scraping {page_url}")
        html_content = get_page(page_url)

        if html_content:
            soup = BeautifulSoup(html_content, 'html.parser')

            # Example: scraping specific data, e.g., all links
            links = soup.find_all('a', href=True)
            for link in links:
                print(link['href'])

            # Check if there's a "next" page (adjust based on the site's structure)
            next_page = soup.find('a', text='Next')  # Adjust selector if necessary
            if next_page:
                page_number += 1
            else:
                print("No more pages to scrape.")
                break
        else:
            break

# Start the scraping process
base_url = "https://gcuf.edu.pk/"  # Replace with the actual base URL of the website
scrape_complete_website(base_url)


Scraping https://gcuf.edu.pk//page/1
Failed to retrieve page: https://gcuf.edu.pk//page/1


In [32]:
import requests
from bs4 import BeautifulSoup

def get_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to retrieve page {url}, status code: {response.status_code}")
            return None
    except requests.RequestException as e:
        print(f"Error fetching page {url}: {e}")
        return None

# Example of scraping a page
base_url = "https://gcuf.edu.pk"
html_content = get_page(base_url)
if html_content:
    soup = BeautifulSoup(html_content, 'html.parser')
    files.download
    print(soup.prettify())  # Print the prettified HTML for inspection
else:
    print("Failed to retrieve content.")


<!DOCTYPE html>
<!--[if IE 8]> <html lang="en" class="ie8"> <![endif]-->
<!--[if IE 9]> <html lang="en" class="ie9"> <![endif]-->
<!--[if !IE]><!-->
<html lang="en">
 <!--<![endif]-->
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Government College University Faisalabad Urdu: گورنمنٹ کالج یونیورسٹی فیصل آباد‬‎) is a general public sector university based in Faisalabad, Pakistan" name="description"/>
  <title>
   GCUF | Government College University Faisalabad
  </title>
  <!-- Meta -->
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="" name="description"/>
  <meta content="" name="author"/>
  <link href="favicon.ico" rel="icon"/>
  <link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,600,700" rel="stylesheet" type="text/css"/>
  <!-- Global CSS -->
  <!--<link rel="stylesheet" href="assets/plugins

In [33]:
import requests
from bs4 import BeautifulSoup
from google.colab import files  # For Google Colab download

# Function to get the HTML content of a webpage
def get_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to retrieve page {url}, status code: {response.status_code}")
            return None
    except requests.RequestException as e:
        print(f"Error fetching page {url}: {e}")
        return None

# Example of scraping a page
base_url = "https://gcuf.edu.pk"
html_content = get_page(base_url)

# If content is fetched, prettify it and save to an HTML file
if html_content:
    soup = BeautifulSoup(html_content, 'html.parser')

    # Saving the prettified HTML content to a file
    file_name = "scraped_page.html"
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(soup.prettify())  # Save the prettified HTML

    # Download the saved HTML file
    files.download(file_name)  # Google Colab function to download files

    print(f"HTML content saved and ready for download: {file_name}")
else:
    print("Failed to retrieve content.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

HTML content saved and ready for download: scraped_page.html
